### PyEmma Featurizer Support

In [1]:
import openpathsampling as paths

In [2]:
#! lazy
import pyemma.coordinates as coor

In [3]:
#! lazy
storage = paths.Storage('trajectory.nc', mode='a')

04-03-16 12:01:10 openpathsampling.netcdfplus.netcdfplus INFO     Open existing netCDF file 'trajectory.nc' for appending - appending existing file


Import a PyEmma Coordinates Module

Using of pyemma featurizers or general other complex code requires a little trick to be storable. Since storing of code only works if we are not dependend on the context (scope) we need to wrap the construction of our featurizer in a function, that gets all it needs from the global scope as a parameter

In [4]:
def pyemma_generator(f):
    f.add_inverse_distances(f.pairs(f.select_Backbone()))

In [5]:
cv = paths.collectivevariable.CV_PyEMMA_Featurizer('pyemma', pyemma_generator, topology=storage.template.topology)

Now use this featurizer generating function to build a collective variable out of it. All we need for that is a name as usual, the generating function, the list of parameters - here only the topology and at best a test snapshot, a template.

In [6]:
cv(storage.trajectories[0]);

Let's save it to the storage

In [7]:
print storage.save(cv)

(store.cvs[CollectiveVariable], 6, 6)


and apply the featurizer to a trajectory

In [8]:
cv(storage.trajectories[2]);

Sync to make sure the cache is written to the netCDF file.

In [9]:
storage.sync_all()

In [10]:
cv(storage.snapshots.all());

In [11]:
storage.cvs.sync()

In [12]:
storage.sync_all()

In [13]:
py_cv = storage.cvs['pyemma']

In [14]:
nc_var = storage.variables['snapshots_cv_%d_values_fw' % storage.idx(py_cv)]

In [15]:
assert(nc_var.shape[1] == 15)
print nc_var.shape[1]

15


In [16]:
assert(nc_var.var_type == 'numpy.float32')
print nc_var.var_type

numpy.float32


In [17]:
#! ignore
print storage.variables['cvs_json'][:]

[ u'{"_cls": "CV_Function", "_dict": {"name": "psi2", "f": {"_marshal": "YwMAAAADAAAAAwAAAEMAAABzHAAAAHwBAHwAAIMBAGQBABN8AgB8AACDAQBkAQATF1MoAgAAAE5pAgAAACgAAAAAKAMAAAB0BAAAAGl0ZW10AwAAAHBzaXQDAAAAcGhpKAAAAAAoAAAAAHMeAAAAPGlweXRob24taW5wdXQtOC00NGFmZmMyZGY1MzU+dAcAAABwcDJfZm5jAQAAAHMCAAAAAAE=", "_module_vars": [], "_global_vars": []}, "cv_store_cache": true, "cv_time_reversible": false, "cv_wrap_numpy_array": false, "cv_requires_lists": false, "cv_scalarize_numpy_singletons": false, "kwargs": {"phi": {"_obj": "cvs", "_idx": 1}, "psi": {"_obj": "cvs", "_idx": 2}}}}'
 u'{"_cls": "CV_MDTraj_Function", "_dict": {"name": "phi", "f": {"_module": "mdtraj.geometry.dihedral", "_name": "compute_dihedrals"}, "cv_store_cache": true, "cv_time_reversible": true, "cv_wrap_numpy_array": true, "cv_requires_lists": true, "cv_scalarize_numpy_singletons": true, "kwargs": {"indices": [[4, 6, 8, 14]]}}}'
 u'{"_cls": "CV_MDTraj_Function", "_dict": {"name": "psi", "f": {"_module": "mdtraj.geometry.dihedral", 

In [18]:
py_cv_idx = storage.idx(py_cv)
print py_cv_idx
py_emma_feat = storage.vars['cvs_json'][py_cv_idx]

6


In [19]:
erg = py_emma_feat(storage.snapshots);

In [20]:
#! lazy
print erg[0::25,2:4]

[[ 3.03981256  2.99009299]
 [ 3.18913245  3.10045004]
 [ 3.00194454  2.95937991]
 [ 3.0536685   2.96377301]]
